In [19]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import joblib

DATA_DIR = "./data/exercise_recognition" # datasets location


In [20]:
# load all csv files
labels = pd.read_csv(f"{DATA_DIR}/labels.csv")
angles = pd.read_csv(f"{DATA_DIR}/angles.csv")
xyz = pd.read_csv(f"{DATA_DIR}/xyz_distances.csv")
d3 = pd.read_csv(f"{DATA_DIR}/calculated_3d_distances.csv")

print("labels:", labels.shape)
print("angles:", angles.shape)
print("xyz:", xyz.shape)
print("d3:", d3.shape)

labels.head()

labels: (448, 2)
angles: (83922, 9)
xyz: (83922, 50)
d3: (83922, 18)


,vid_id,class
0,0,jumping_jack
1,1,jumping_jack
2,2,jumping_jack
3,3,jumping_jack
4,4,jumping_jack


In [21]:
# merge on vid_id + frame_order
feat = angles.merge(xyz, on=["vid_id", "frame_order"], how="inner")
feat = feat.merge(d3, on=["vid_id", "frame_order"], how="inner")

# numeric feature columns
feature_cols = [c for c in feat.columns if c not in ["vid_id", "frame_order"]]
print("num features:", len(feature_cols))

num features: 71


In [23]:
classes = ["jumping_jack", "push_up", "situp"]
labels_3 = labels[labels["class"].isin(classes)].copy()

classes_3 = sorted(labels_3["class"].unique())
class3_to_id = {c: i for i, c in enumerate(classes_3)}
id_to_class3 = {i: c for c, i in class3_to_id.items()}

labels_3["class_id"] = labels_3["class"].map(class3_to_id)

print("Classes:", classes_3)
print(labels_3["class"].value_counts())

Classes: ['jumping_jack', 'push_up', 'situp']
class
jumping_jack    107
push_up          99
situp            78
Name: count, dtype: int64


In [31]:
def summarize_video(vid_id: int) -> np.ndarray:
    df_v = feat[feat["vid_id"] == vid_id].sort_values("frame_order")
    arr = df_v[feature_cols].to_numpy()  # [T, F]

    mean = arr.mean(axis=0)
    std  = arr.std(axis=0)
    minv = arr.min(axis=0)
    maxv = arr.max(axis=0)
    return np.concatenate([mean, std, minv, maxv])

vid_ids = labels_3["vid_id"].values

X_all = []
y_all = []

for vid in vid_ids_3:
    X_all.append(summarize_video(vid))
    cls_id = int(labels_3.loc[labels_3["vid_id"] == vid, "class_id"].iloc[0])
    y_all.append(cls_id)

X_all = np.vstack(X_all)
y_all = np.array(y_all)

print("X_all:", X_all.shape)
print("y_all:", y_all.shape)


X_all: (284, 284)
y_all: (284,)


In [32]:
# train/val/test split
idx = np.arange(len(vid_ids_3))

idx_train, idx_temp, y_train, y_temp = train_test_split(
    idx, y_all, test_size=0.3, stratify=y_all, random_state=42
)

idx_val, idx_test, y_val, y_test = train_test_split(
    idx_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

X_train, X_val, X_test = X_all[idx_train], X_all[idx_val], X_all[idx_test]

print("Train vids:", X_train.shape[0])
print("Val vids:", X_val.shape[0])
print("Test vids:", X_test.shape[0])


Train vids: 198
Val vids: 43
Test vids: 43


In [33]:
# MLP classifier
clf3 = make_pipeline(
    StandardScaler(),
    MLPClassifier(
        hidden_layer_sizes=(256, 128, 64),
        activation="relu",
        alpha=1e-4,
        max_iter=800,
        early_stopping=True,
        n_iter_no_change=20,
        random_state=42,
    ),
)

clf3.fit(X_train, y_train)


,steps,"[('standardscaler', ...), ('mlpclassifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,hidden_layer_sizes,"(256, ...)"
,activation,'relu'
,solver,'adam'
,alpha,0.0001


In [34]:
print("VALIDATION PERFORMANCE (3 classes)")
y_val_pred = clf3.predict(X_val)
print(classification_report(y_val, y_val_pred, target_names=classes_3))

VALIDATION PERFORMANCE (3 classes)
              precision    recall  f1-score   support

jumping_jack       1.00      1.00      1.00        16
     push_up       1.00      1.00      1.00        15
       situp       1.00      1.00      1.00        12

    accuracy                           1.00        43
   macro avg       1.00      1.00      1.00        43
weighted avg       1.00      1.00      1.00        43



In [35]:
print("TEST PERFORMANCE (3 classes)")
y_test_pred = clf3.predict(X_test)
print(classification_report(y_test, y_test_pred, target_names=classes_3))

TEST PERFORMANCE (3 classes)
              precision    recall  f1-score   support

jumping_jack       1.00      1.00      1.00        16
     push_up       1.00      1.00      1.00        15
       situp       1.00      1.00      1.00        12

    accuracy                           1.00        43
   macro avg       1.00      1.00      1.00        43
weighted avg       1.00      1.00      1.00        43



In [36]:
joblib.dump(clf3, "exercise_type_3class_mlp.pkl")
joblib.dump(classes_3, "exercise_type_3class_labels.pkl")
print("Saved 3-class model + labels.")

Saved 3-class model + labels.
